In [94]:
%cd /mnt/f/oss_p2

/mnt/f/oss_p2


/home/coldbrew/anaconda3/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [95]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix

# 데이터 로드 
ratings = pd.read_csv('/mnt/f/oss_p2/ml-1m/ratings.dat', sep='::', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'], engine='python')

ratings.drop(['Timestamp'], axis=1, inplace=True)
#ratings = ratings.pivot(index='UserID', columns='MovieID', values='Rating')
ratings

,UserID,MovieID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [96]:
rating_pivot = ratings.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)
rating_pivot

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 1st KMeans clustering


In [97]:
# NumPy 배열로 변환
user_item_matrix_np = rating_pivot.to_numpy()

# K-Means 클러스터링
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(user_item_matrix_np)

# 클러스터 레이블 출력
labels = kmeans.labels_

# 각 사용자별 클러스터 레이블을 데이터프레임으로 저장
user_clusters = pd.DataFrame({'UserID': rating_pivot.index, 'Cluster': labels})

print(user_clusters)

# 클러스터 레이블을 원본 데이터프레임에 추가
ratings = ratings.merge(user_clusters, on='UserID')

# 각 클러스터별 통계 출력
print(ratings.groupby('Cluster').mean())

      UserID  Cluster
0          1        1
1          2        2
2          3        1
3          4        1
4          5        1
...      ...      ...
6035    6036        0
6036    6037        2
6037    6038        1
6038    6039        1
6039    6040        2

[6040 rows x 2 columns]
              UserID      MovieID    Rating
Cluster                                    
0        2959.827615  1872.132363  3.485515
1        2998.955378  1905.394144  3.675809
2        3100.108614  1834.045868  3.609390


## 2nd


### Group Recommender Algorithms


In [98]:
# 클러스터가 1인것 만 선택
cluster1 = ratings[ratings['Cluster'] == 0]
cluster1_pivot = cluster1.pivot(index='UserID', columns='MovieID', values='Rating')
print(cluster1_pivot.shape)
# 클러스터가 2인것 만 선택
cluster2 = ratings[ratings['Cluster'] == 1]
cluster2_pivot = cluster2.pivot(index='UserID', columns='MovieID', values='Rating')
print(cluster2_pivot.shape)
# 클러스터가 3인것 만 선택
cluster3 = ratings[ratings['Cluster'] == 2]
cluster3_pivot = cluster3.pivot(index='UserID', columns='MovieID', values='Rating')
print(cluster3_pivot.shape)


(568, 3565)
(3808, 3516)
(1664, 3562)


In [99]:
def print_reco_by_algo(cluster):
    cluster_pivot_AU = cluster.sum(axis=0).sort_values(ascending=False).head(10)
    print("**Additive Utilitarian : \n", cluster_pivot_AU)
    cluster_pivot_AVG = cluster.mean(axis=0).sort_values(ascending=False).head(10)
    print("**Average : \n", cluster_pivot_AVG)
    cluster_pivot_SC = cluster[cluster > 0].count(axis=0).sort_values(ascending=False).head(10)
    print("**Simple Count : \n", cluster_pivot_SC)
    cluster_pivot_AV = cluster[cluster > 4].count(axis=0).sort_values(ascending=False).head(10)
    print("**Approval Voting : \n", cluster_pivot_AV)

    ranks = cluster.rank(axis=1, method='average' )-1
    borda_aggregated = ranks.sum(axis=0)
    borda_aggregated_series = pd.Series(borda_aggregated, index=cluster.columns)
    print("**Borda Voting : \n", borda_aggregated_series.sort_values(ascending=False).head(10))
    
    # Copeland Rule 구현
    num_items = cluster.shape[1]

    user_item_matrix_np = cluster.values
    items = cluster.columns
    comparison_matrix = np.zeros((num_items, num_items), dtype=int)

    # 넘파이 벡터연산으로 빠르게 연산 수행, 기존 방법은 매우 느려서 사용 불가
    for i in range(num_items):
        for j in range(i + 1, num_items):
            item1_scores = user_item_matrix_np[:, i]
            item2_scores = user_item_matrix_np[:, j]
            
            item1_wins = np.sum(item1_scores > item2_scores)
            item2_wins = np.sum(item1_scores < item2_scores)
            
            if item1_wins > item2_wins:
                comparison_matrix[i, j] = -1
                comparison_matrix[j, i] = 1
            elif item2_wins > item1_wins:
                comparison_matrix[i, j] = 1
                comparison_matrix[j, i] = -1

    # 결과를 DataFrame으로 변환
    comparison_df = pd.DataFrame(comparison_matrix, index=items, columns=items)
    copeland_rule = comparison_df.sum(axis=0)

    print("**Copeland Rule: \n", copeland_rule.sort_values(ascending=False).head(10))

print("------------Cluster 1------------")
print_reco_by_algo(cluster1_pivot)
print("------------Cluster 2------------")
print_reco_by_algo(cluster2_pivot)
print("------------Cluster 3------------")
print_reco_by_algo(cluster3_pivot)

------------Cluster 1------------
**Additive Utilitarian : 
 MovieID
260     2443.0
1196    2422.0
1198    2360.0
1270    2216.0
608     2194.0
593     2175.0
296     2169.0
858     2146.0
2571    2144.0
1197    2135.0
dtype: float64
**Average : 
 MovieID
696     5.000000
666     5.000000
3232    5.000000
981     5.000000
572     5.000000
787     5.000000
853     5.000000
2538    5.000000
3338    4.916667
858     4.675381
dtype: float64
**Simple Count : 
 MovieID
1196    548
260     535
1270    534
1580    519
1210    515
1097    514
2716    514
1198    510
589     506
480     504
dtype: int64
**Approval Voting : 
 MovieID
260     360
1198    352
858     343
1196    304
296     300
318     299
2858    293
608     287
527     284
593     284
dtype: int64
**Borda Voting : 
 MovieID
260     267802.0
1198    263294.0
1196    259505.0
593     243136.0
858     242095.0
608     240256.0
296     237532.0
2858    233807.0
318     233091.5
2571    231276.0
dtype: float64
**Copeland Rule: 
 Movie

In [100]:
user_item_matrix = pd.DataFrame({
    'i1': [2, 3, 2],
    'i2': [3, 2, np.nan],
    'i3': [5, 5, 1],
    'i4': [4, 4, 3],
    'i5': [np.nan, 3, 4]
}, index=['u1', 'u2', 'u3'])
data = {
    "i1": [8, 7, 5],
    "i2": [10, 10, 1],
    "i3": [7, 6, 8],
    "i4": [10, 9, 6],
    "i5": [9, 8, 9],
    "i6": [8, 10, 10],
    "i7": [10, 9, 3],
    "i8": [6, 4, 5],
    "i9": [3, 4, 7],
    "i10": [6, 7, 10]
}
df = pd.DataFrame(data, index=["U1", "U2", "U3"])
print(df)
# pandas를 이용한 Borda 순위 계산
ranks = df.rank(axis=1, method='average', )-1
print(ranks)

borda_aggregated = ranks.sum(axis=0)
borda_aggregated_series = pd.Series(borda_aggregated, index=df.columns)


# 결과 출력
print("**Borda Voting : \n", borda_aggregated_series.sort_values(ascending=False).head(10))
print(print_reco_by_algo(df))


    i1  i2  i3  i4  i5  i6  i7  i8  i9  i10
U1   8  10   7  10   9   8  10   6   3    6
U2   7  10   6   9   8  10   9   4   4    7
U3   5   1   8   6   9  10   3   5   7   10
     i1   i2   i3   i4   i5   i6   i7   i8   i9  i10
U1  4.5  8.0  3.0  8.0  6.0  4.5  8.0  1.5  0.0  1.5
U2  3.5  8.5  2.0  6.5  5.0  8.5  6.5  0.5  0.5  3.5
U3  2.5  0.0  6.0  4.0  7.0  8.5  1.0  2.5  5.0  8.5
**Borda Voting : 
 i6     21.5
i4     18.5
i5     18.0
i2     16.5
i7     15.5
i10    13.5
i3     11.0
i1     10.5
i9      5.5
i8      4.5
dtype: float64
**Additive Utilitarian : 
 i6     28
i5     26
i4     25
i10    23
i7     22
i2     21
i3     21
i1     20
i8     15
i9     14
dtype: int64
**Average : 
 i6     9.333333
i5     8.666667
i4     8.333333
i10    7.666667
i7     7.333333
i2     7.000000
i3     7.000000
i1     6.666667
i8     5.000000
i9     4.666667
dtype: float64
**Simple Count : 
 i1     3
i2     3
i3     3
i4     3
i5     3
i6     3
i7     3
i8     3
i9     3
i10    3
dtype: int64
**Appro

## Copeland Rule 연습


In [101]:
import numpy as np
import pandas as pd


user_item_matrix = pd.DataFrame({
'i1': [2, 3, 2],
'i2': [3, 2, np.nan],
'i3': [5, 5, 1],
'i4': [4, 4, 3],
'i5': [np.nan, 3, 4]
}, index=['u1', 'u2', 'u3']).fillna(0)
data = {
    "i1": [8, 7, 5],
    "i2": [10, 10, 1],
    "i3": [7, 6, 8],
    "i4": [10, 9, 6],
    "i5": [9, 8, 9],
    "i6": [8, 10, 10],
    "i7": [10, 9, 3],
    "i8": [6, 4, 5],
    "i9": [3, 4, 7],
    "i10": [6, 7, 10]
}
# Assign user labels to the DataFrame
df = pd.DataFrame(data, index=["U1", "U2", "U3"])

user_item_matrix= df
items = user_item_matrix.columns
scores = {item: 0 for item in items}

# 비교 행렬 초기화
comparison_matrix = pd.DataFrame(index=items, columns=items, data=0)

for i in range(1, len(items)+1):
    for j in range(1,len(items)+1):
        if i != j and i < j:
            print(i, j)
            # item i 와 item j 중 높은 점수를 받은 유저가 많으면 승점을 기록한다.
            item1_wins = 0
            item2_wins = 0
            for user in user_item_matrix.index:
                #print(user, "에 대해서 ")
                if user_item_matrix.at[user, items[i-1]] > user_item_matrix.at[user, items[j-1]]:
                    #print(items[i-1]," > " ,items[j-1])
                    item1_wins += 1
                elif user_item_matrix.at[user, items[i-1]] < user_item_matrix.at[user, items[j-1]]:
                    #print(items[i-1]," < " ,items[j-1])
                    item2_wins += 1
            print("선호하는 유저가 많은 아이템은 ? ",item1_wins, item2_wins)
            if item1_wins > item2_wins:
                comparison_matrix.at[items[i-1], items[j-1]] = -1
                comparison_matrix.at[items[j-1], items[i-1]] = 1
            elif item2_wins > item1_wins:
                comparison_matrix.at[items[i-1], items[j-1]] = 1
                comparison_matrix.at[items[j-1], items[i-1]] = -1

print(scores)      
print(comparison_matrix)
copeland_rule = comparison_matrix.sum(axis=0)
copeland_rule

1 2
선호하는 유저가 많은 아이템은 ?  1 2
1 3
선호하는 유저가 많은 아이템은 ?  2 1
1 4
선호하는 유저가 많은 아이템은 ?  0 3
1 5
선호하는 유저가 많은 아이템은 ?  0 3
1 6
선호하는 유저가 많은 아이템은 ?  0 2
1 7
선호하는 유저가 많은 아이템은 ?  1 2
1 8
선호하는 유저가 많은 아이템은 ?  2 0
1 9
선호하는 유저가 많은 아이템은 ?  2 1
1 10
선호하는 유저가 많은 아이템은 ?  1 1
2 3
선호하는 유저가 많은 아이템은 ?  2 1
2 4
선호하는 유저가 많은 아이템은 ?  1 1
2 5
선호하는 유저가 많은 아이템은 ?  2 1
2 6
선호하는 유저가 많은 아이템은 ?  1 1
2 7
선호하는 유저가 많은 아이템은 ?  1 1
2 8
선호하는 유저가 많은 아이템은 ?  2 1
2 9
선호하는 유저가 많은 아이템은 ?  2 1
2 10
선호하는 유저가 많은 아이템은 ?  2 1
3 4
선호하는 유저가 많은 아이템은 ?  1 2
3 5
선호하는 유저가 많은 아이템은 ?  0 3
3 6
선호하는 유저가 많은 아이템은 ?  0 3
3 7
선호하는 유저가 많은 아이템은 ?  1 2
3 8
선호하는 유저가 많은 아이템은 ?  3 0
3 9
선호하는 유저가 많은 아이템은 ?  3 0
3 10
선호하는 유저가 많은 아이템은 ?  1 2
4 5
선호하는 유저가 많은 아이템은 ?  2 1
4 6
선호하는 유저가 많은 아이템은 ?  1 2
4 7
선호하는 유저가 많은 아이템은 ?  1 0
4 8
선호하는 유저가 많은 아이템은 ?  3 0
4 9
선호하는 유저가 많은 아이템은 ?  2 1
4 10
선호하는 유저가 많은 아이템은 ?  2 1
5 6
선호하는 유저가 많은 아이템은 ?  1 2
5 7
선호하는 유저가 많은 아이템은 ?  1 2
5 8
선호하는 유저가 많은 아이템은 ?  3 0
5 9
선호하는 유저가 많은 아이템은 ?  3 0
5 10
선호하는 유저가 많은 아이템은 ?  2 1
6 7
선호하는 유저가 많은

i1    -2
i2     6
i3    -5
i4     6
i5     1
i6     8
i7     4
i8    -8
i9    -8
i10   -2
dtype: int64

In [102]:
import numpy as np
import pandas as pd

user_item_matrix = pd.DataFrame(data).fillna(0)

items = user_item_matrix.columns
num_items = len(items)
scores = {item: 0 for item in items}

# numpy 배열
user_item_matrix_np = user_item_matrix.values
comparison_matrix = np.zeros((num_items, num_items), dtype=int)

# 기존 알고리즘
for i in range(num_items):
    for j in range(i + 1, num_items):
        item1_wins = 0
        item2_wins = 0
        for user_ratings in user_item_matrix_np:
            if user_ratings[i] > user_ratings[j]:
                item1_wins += 1
            elif user_ratings[i] < user_ratings[j]:
                item2_wins += 1
        if item1_wins > item2_wins:
            comparison_matrix[i, j] = -1
            comparison_matrix[j, i] = 1
        elif item2_wins > item1_wins:
            comparison_matrix[i, j] = 1
            comparison_matrix[j, i] = -1

# 결과를 DataFrame으로 변환
comparison_df = pd.DataFrame(comparison_matrix, index=items, columns=items)
copeland_rule = comparison_df.sum(axis=0)

print(comparison_df)
print(copeland_rule.head(10))

     i1  i2  i3  i4  i5  i6  i7  i8  i9  i10
i1    0   1  -1   1   1   1   1  -1  -1    0
i2   -1   0  -1   0  -1   0   0  -1  -1   -1
i3    1   1   0   1   1   1   1  -1  -1    1
i4   -1   0  -1   0  -1   1  -1  -1  -1   -1
i5   -1   1  -1   1   0   1   1  -1  -1   -1
i6   -1   0  -1  -1  -1   0  -1  -1  -1   -1
i7   -1   0  -1   1  -1   1   0  -1  -1   -1
i8    1   1   1   1   1   1   1   0   0    1
i9    1   1   1   1   1   1   1   0   0    1
i10   0   1  -1   1   1   1   1  -1  -1    0
i1    -2
i2     6
i3    -5
i4     6
i5     1
i6     8
i7     4
i8    -8
i9    -8
i10   -2
dtype: int64


In [103]:
data = {
    "i1": [8, 7, 5],
    "i2": [10, 10, 1],
    "i3": [7, 6, 8],
    "i4": [10, 9, 6],
    "i5": [9, 8, 9],
    "i6": [8, 10, 10],
    "i7": [10, 9, 3],
    "i8": [6, 4, 5],
    "i9": [3, 4, 7],
    "i10": [6, 7, 10]
}
# Assign user labels to the DataFrame
df = pd.DataFrame(data, index=["U1", "U2", "U3"])
cluster2_pivot = df
num_items = cluster2_pivot.shape[1]
num_users = cluster2_pivot.shape[0]

# Convert the DataFrame to numpy array for faster computation
user_item_matrix_np = cluster2_pivot.values
items = cluster2_pivot.columns
comparison_matrix = np.zeros((num_items, num_items), dtype=int)

#넘파이 벡터연산으로 빠르게 연산 수행 기존 방법은 매우 느려서 사용 불가
for i in range(num_items):
    for j in range(i + 1, num_items):
        item1_scores = user_item_matrix_np[:, i]
        item2_scores = user_item_matrix_np[:, j]
        
        item1_wins = np.sum(item1_scores > item2_scores)
        item2_wins = np.sum(item1_scores < item2_scores)
        
        if item1_wins > item2_wins:
            comparison_matrix[i, j] = -1
            comparison_matrix[j, i] = 1
        elif item2_wins > item1_wins:
            comparison_matrix[i, j] = 1
            comparison_matrix[j, i] = -1

# 결과를 DataFrame으로 변환
comparison_df = pd.DataFrame(comparison_matrix, index=items, columns=items)
copeland_rule = comparison_df.sum(axis=0)

print(comparison_df)
print(copeland_rule.sort_values(ascending=False).head(10))

     i1  i2  i3  i4  i5  i6  i7  i8  i9  i10
i1    0   1  -1   1   1   1   1  -1  -1    0
i2   -1   0  -1   0  -1   0   0  -1  -1   -1
i3    1   1   0   1   1   1   1  -1  -1    1
i4   -1   0  -1   0  -1   1  -1  -1  -1   -1
i5   -1   1  -1   1   0   1   1  -1  -1   -1
i6   -1   0  -1  -1  -1   0  -1  -1  -1   -1
i7   -1   0  -1   1  -1   1   0  -1  -1   -1
i8    1   1   1   1   1   1   1   0   0    1
i9    1   1   1   1   1   1   1   0   0    1
i10   0   1  -1   1   1   1   1  -1  -1    0
i6     8
i2     6
i4     6
i7     4
i5     1
i1    -2
i10   -2
i3    -5
i8    -8
i9    -8
dtype: int64


In [104]:
copeland_rule.sort_values(ascending=False).head(10)

i6     8
i2     6
i4     6
i7     4
i5     1
i1    -2
i10   -2
i3    -5
i8    -8
i9    -8
dtype: int64